# <font color='crimson'>1. 모듈 가져오기</font>

In [9]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

---
<br>

# <font color='crimson'>2. 클래스 레이블 가져오기</font>
#### 예측 결과 확인용

In [10]:
with open('coco_class_names.txt', 'r') as f:
    class_names = f.read().split('\n')

---
<br>

# <font color='crimson'>3. 이미지 가져오기 & 예측용으로 설정</font>

### <font color='darkcyan'>이미지 가져오기 Modules</font>

In [21]:
img_url = '/home/jetbot/works/jetson-inference/build/aarch64/bin/images/'
image = cv2.imread(img_url + 'peds_0.jpg')
image_height, image_width, _ = image.shape


### <font color='darkcyan'>Blob(Binary Lage Object) 형태로 변환</font>

In [22]:
blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), swapRB=True)

---
<br>

# <font color='crimson'>4. 예측</font>

### <font color='darkcyan'>모델 불러오기 & 입력 설정</font>

In [23]:
model = cv2.dnn.readNet(model='frozen_inference_graph.pb', config='ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',framework='TensorFlow')
model.setInput(blob)

### <font color='darkcyan'>예측</font>

In [24]:
output = model.forward()

### <font color='darkcyan'>예측 결과 편집(박스, 텍스트 입력)</font>

In [25]:
for detection in output[0, 0, :, :]:

    confidence = detection[2]
    if confidence > .4:
        class_id = detection[1]
        class_name = class_names[int(class_id)-1]

        box_x = detection[3] * image_width
        box_y = detection[4] * image_height
        box_width = detection[5] * image_width
        box_height = detection[6] * image_height

        cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), (0,255,0), thickness=2)
        cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

### <font color='darkcyan'>결과 출력(박스, 텍스트 입력)</font>

In [26]:
image = bgr8_to_jpeg(image)  #numpy를 이미지로 변환
widget = widgets.Image(format='jpeg', width=500, height=500)
widget.value = image
display(widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

---
<br>

# <font color='crimson'>5. 동영상 예측</font>

### <font color='darkcyan'>이미지 출력 위젯 생성</font>

In [29]:
widget = widgets.Image(format='jpeg', width=500, height=500)
display(widget)

Image(value=b'', format='jpeg', height='500', width='500')

### <font color='darkcyan'>동영상 가져오기</font>

In [30]:
cap = cv2.VideoCapture('/usr/share/visionworks/sources/data/pedestrians.mp4')
if cap.isOpened():
    while True:
        ret, image = cap.read()
        if not ret:
            break

        image_height, image_width, _ = image.shape

        blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(0,0,0), swapRB=True)

        model.setInput(blob)

        output = model.forward()

        for detection in output[0, 0, :, :]:
            confidence = detection[2]
            if confidence > .2:
                class_id = detection[1]

                class_name = class_names[int(class_id)-1]

                box_x = detection[3] * image_width
                box_y = detection[4] * image_height

                box_width = detection[5] * image_width
                box_height = detection[6] * image_height

                cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), (0,255,0), thickness=2)
                cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        image = bgr8_to_jpeg(image)
        widget.value = image   


KeyboardInterrupt: 